In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statistics
from pathlib import Path
import datetime

import requests
import re
import os
import sys


from multiprocess import Pool
import psutil
from tqdm import tqdm

from bs4 import BeautifulSoup

import glob

import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups.csv.gzip', compression = 'gzip')#, nrows = 100000 )

/tmp/ipykernel_1060181/1600029537.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups.csv.gzip', compression = 'gzip')#, nrows = 100000 )


In [ ]:
data

In [18]:
data['posting_date'] = pd.to_datetime(data['posting_date'], utc=True)

In [14]:
posting_frequency = data.groupby("day").count()
posting_frequency = posting_frequency[["posting_date"]]

In [15]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe'

fig = px.line(posting_frequency, x=posting_frequency.index, y='posting_date')
fig.show()

In [19]:
ig = px.line(posting_frequency, x=posting_frequency.index, y='posting_date')
fig.show()

In [50]:
data.head(10)

,Unnamed: 0,uid,mid_message,posting_date,message,fwd_message,fwd_posting_date_message,mid_file,posting_date_file,link_url,...,weekday,message_hash,fwd_message_hash,website,message_id,replied_to,transcribed_message,year,month,day_of_year
0,0,44956832152488193039965,4495.0,2020-04-05 10:03:27+00:00,"<strong>‼️‼️❤️‼️‼️❤️‼️‼️ BITTE AN OLIVER JANICH WEITERLEITEN! </strong>Lieber Oliver, danke für dieses gutes Video von dir💓 endlich sagt es mal jemand öffentlich das dieser Kanal „unzensiert“ sehr viele Lügen verbreitet sowie sehr viele negative Dinge über bestimmte Menschen .. oder Menschengruppen !!! Ich habe vor diesem Kanal schon lange gewarnt!! Jetzt endlich sagst auch Du es öffentlich ❤️🙏❤️ .. kommt am bei Minute 30:00 des Videos <a href=""https://youtu.be/4-6CwDPwmTE"">https://youtu.be/4-6CwDPwmTE</a> <a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,NaN,4495.0,2020-04-05 10:03:27+00:00,https://youtu.be/4-6CwDPwmTE,...,7.0,9.010954e+18,NaN,youtu.be,4495.0,NaN,NaN,2020.0,4.0,96.0
1,1,44976832152488193039965,4497.0,2020-04-05 11:12:57+00:00,"‼️‼️ACHTUNG‼️‼️ <strong> Satanisten Seite mit Fotos von Jay- Z </strong> Satanistischer Illuminati Order .. sie sagen sie würden helfen .. eine bessere Welt aufzubauen .. ihre Fotos sagen jedoch etwas Anderes !!! Dort gibt es auch Fotos von diesem Rapper Jay - Z auf dieser Seite ... <a href=""https://illuminatireligion.com/our-mission/"">https://illuminatireligion.com/our-mission/</a> <a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,NaN,4497.0,2020-04-05 11:12:57+00:00,https://illuminatireligion.com/our-mission/,...,7.0,2.006496e+18,NaN,illuminatireligion.com,4497.0,NaN,NaN,2020.0,4.0,96.0
2,2,44986832152488193039965,4498.0,2020-04-05 11:26:07+00:00,"<a href=""https://qlobal-change.blogspot.com/2020/04/putin-trump-gegen-die-neue-weltordnung.html?m=1"">https://qlobal-change.blogspot.com/2020/04/putin-trump-gegen-die-neue-weltordnung.html?m=1</a> <a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,NaN,4498.0,2020-04-05 11:26:07+00:00,https://qlobal-change.blogspot.com/2020/04/putin-trump-gegen-die-neue-weltordnung.html?m=1,...,7.0,9.522348e+16,NaN,qlobal-change.blogspot.com,4498.0,NaN,NaN,2020.0,4.0,96.0
3,3,44996832152488193039965,4499.0,2020-04-05 11:27:18+00:00,"<a href=""https://youtu.be/dkklDuUUGzw"">https://youtu.be/dkklDuUUGzw</a> <a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,NaN,4499.0,2020-04-05 11:27:18+00:00,https://youtu.be/dkklDuUUGzw,...,7.0,-1.990757e+18,NaN,youtu.be,4499.0,NaN,NaN,2020.0,4.0,96.0
4,4,45006832152488193039965,4500.0,2020-04-05 11:52:12+00:00,"<a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,NaN,4500.0,2020-04-05 11:52:12+00:00,NaN,...,7.0,6.824871e+18,NaN,NaN,4500.0,NaN,NaN,2020.0,4.0,96.0
5,5,45016832152488193039965,4501.0,2020-04-05 11:52:12+00:00,"<a href=""https://t.me/Geheimnis_Gesundheit"">@Geheimnis_Gesundheit</a> <a href=""https://t.me/s/Geheimnis_Gesundheit"">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar Geheimnis der Gesundheit eure 😇 Angel of Truth",NaN,

In [20]:
data["day"] = data["posting_date"].dt.floor('D') + pd.Timedelta(12, unit='h')

In [21]:
data["year"] = data["posting_date"].dt.year

In [22]:
data["month"] = data["posting_date"].dt.month

In [23]:
data["week"] = data["posting_date"].dt.isocalendar().week

In [24]:
data["day_of_year"] = data["posting_date"].dt.dayofyear

In [25]:
def link_removal(some_string):
    some_string = re.sub("<a.*?a>", "", some_string)
    some_string = re.sub("<code>.*?</code>", "", some_string)
    some_string = re.sub("<.*?>", "", some_string)

    return some_string

In [26]:
def get_date(day, year):

    date = datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1)
    
    return date

In [27]:
def get_date_of_first_weekday(week, year):
 
    date = datetime.datetime.strptime(f'{int(year)}-W{int(week)}-1', '%G-W%V-%u').date()
    
    return date

In [28]:
def get_date_of_first_monthday(month, year):

    date = datetime.datetime(int(year), int(month), 1).date()
    
    return date

In [29]:
def first_day_of_denominator_as_datetime(part, year, denominator):
    
    if denominator == 'day_of_year':
        return get_date(part, year)
    elif denominator == 'week':
        return get_date_of_first_weekday(part, year)
    elif denominator == 'month':
        return get_date_of_first_monthday(part, year)
    else:
        #dummy date to filter out
        return datetime.datetime(1970, 1, 1).date()
    

In [30]:
#function to create a df with concatenated text documents
#chats from each day, week, or month can be concatenated to a text document

def create_text_df(denominator, df):
    
    text_df = pd.DataFrame(columns=["group_name", "denominator", "year", "messages", "date"]) # Add if working with whisper transcriptions , "transcribed_message"])

    group_names = list(df["group_name"].dropna().unique())#[3:4]
    denom = list(df[denominator].dropna().unique())#[:20]
    year = list(df['year'].dropna().unique())#[1:2]

    for g in tqdm(group_names):
        group = g
        for y in year:
            for d in denom:
                build_text = df.loc[(df["group_name"] == g) & (df[denominator] == d) & (df["year"] == y)]             
                if len(build_text) > 0: 
                    
                    date = first_day_of_denominator_as_datetime(d, y, denominator)
                    
                    build_text.loc[:, "message"] = build_text["message"].fillna("")
                    #build_text.loc[:, "message"] = build_text["message"].apply(link_removal)
                    build_text.loc[:, "fwd_message"] = build_text["fwd_message"].fillna("")
                    #build_text.loc[:, "fwd_message"] = build_text["fwd_message"].apply(link_removal)
                    #build_text["transcribed_message"] = build_text["transcribed_message"].fillna("")
                    #build_text["transcribed_message"] = build_text["transcribed_message"].apply(link_removal)
                    messages = " ".join(build_text["message"].to_list())+" " + " ".join(build_text["fwd_message"].to_list()) # + " ".join(build_text["transcribed_message"].to_list())

                    new_row = len(text_df)
                    #print(group, d, y, date)
                    
                    text_df.loc[new_row] = [group, d, y, messages, date]
                    
                    

    return text_df

    #except: 
    #    return 'Type in valid denominator as string. Valid denominators are day_of_year, week, month'

In [31]:
text_df = create_text_df('month', data)

100%|███████████████████████████████████████████████████████████████████████████████████| 26/26 [00:12<00:00,  2.05it/s]


In [32]:
text_df

,group_name,denominator,year,messages,date
0,🍃🌺 GᕮᕼᕮIᗰᑎIS 💫 ᗪᕮᖇ GᕮSᑌᑎᗪᕼᕮIT 🌺🍃,4.0,2020.0,‼️‼️❤️‼️‼️❤️‼️‼️ BITTE AN OLIVER JANICH WEITE...,2020-04-01
1,USWTV - Videoarchiv,5.0,2021.0,2036 - Das Ende der reptiloiden Schreckensherr...,2021-05-01
2,Stop-BRD Chat,9.0,2020.0,Danke ich schaue es mir noch mal in Ruhe an. V...,2020-09-01
3,Stop-BRD Chat,8.0,2020.0,Was haltet ihr davon ? 🙌☺️👍 Wi...,2020-08-01
4,Stop-BRD Chat,7.0,2020.0,"Hallo alle👋🏼 Hallo Hey, prima k...",2020-07-01
...,...,...,...,...,...
171,GEMEINSAMER ANTI CORONALÜGEN - SIEG CHAT! ZUM ...,5.0,2021.0,🎼 👊✊ Hallo 🔴 Aufkleber zum ausdr...,2021-05-01
172,🧩 SPANDAU steht AUF!,4.0,2021.0,hab hier auf arbeit gerade meinen spaß mit den...,2021-04-01
173,🧩 SPANDAU steht AUF!,5.0,2021.0,Ich habe tatsächlich schon Angebote zu gefälsc...,2021-05-01
174,🧩 SPANDAU steht AUF!,6.0,2021.0,❗️❗️❗️ HILFE BENÖTIGT ❗️❗️Hat einer von euch L...,2021-06-01


In [6]:
#use only with large dataset to get a randomized sub-sample
#######################################!!!########################################
'''
import random

random.seed(42)

groups = data["group_name"].dropna().unique().tolist()
random_groups = random.sample(groups, 10)

sampled_df = text_df[text_df["group_name"].isin(random_groups)].reset_index()

doc_list = sampled_df["messages"].to_list()
date_list = sampled_df["day"] +.to_list()
'''
########################################!!!#######################################

'\nimport random\n\nrandom.seed(42)\n\ngroups = data["group_name"].dropna().unique().tolist()\nrandom_groups = random.sample(groups, 10)\n\nsampled_df = text_df[text_df["group_name"].isin(random_groups)].reset_index()\n\ndoc_list = sampled_df["messages"].to_list()\ndate_list = sampled_df["day"] +.to_list()\n'

In [33]:
text_df.date

0      2020-03-30
1      2021-05-03
2      2020-09-21
3      2020-08-17
4      2020-08-24
          ...    
285    2021-08-02
286    2021-09-13
287    2021-09-20
288    2021-09-06
289    2021-10-11
Name: date, Length: 290, dtype: object

In [79]:
doc_list = text_df["messages"].tolist()
date_list = text_df["date"].apply(lambda x: x.strftime("%d-%m-%Y %H:%M:%S")).tolist()

In [80]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from bertopic import BERTopic

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

german_stop_words = stopwords.words('german')

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words= german_stop_words)
umap_model = UMAP(n_neighbors=10, n_components =2, metric="cosine", min_dist=0.0)
hdbscan_model = HDBSCAN(min_cluster_size=5, metric="euclidean", prediction_data=True)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mangermaier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
topic_model = BERTopic(language="german",
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       umap_model=umap_model,
                       calculate_probabilities=False,
                       verbose=True)

In [82]:
date_list[0]

'01-04-2020 00:00:00'

In [83]:
topics = topic_model.fit_transform(doc_list)

Batches:   0%|          | 0/58 [00:00<?, ?it/s]

2023-04-20 15:04:22,208 - BERTopic - Transformed documents to Embeddings
2023-04-20 15:04:35,143 - BERTopic - Reduced dimensionality
2023-04-20 15:04:35,218 - BERTopic - Clustered reduced embeddings


In [84]:
hierarchical_topics = topic_model.hierarchical_topics(doc_list)
vis_topics = topic_model.visualize_hierarchical_documents(doc_list, hierarchical_topics)

100%|███████████████████████████████████████████████████████████████████████████████████| 72/72 [01:34<00:00,  1.31s/it]


In [ ]:
topics_over_time = topic_model.topics_over_time(doc_list, date_list, nr_bins=20)

5it [25:45, 473.92s/it]

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [ ]:
vis_topics

In [231]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,174,-1_at_ja_mal_schon
1,0,51,0_red rainbow_rainbow_red_
2,1,24,1_ja_danke_mal_axel
3,2,23,2_dankbar_dankbarkeit_wofür_freude
4,3,21,3_the_and_to_you
5,4,20,4_deutschland_mehr_spd_000
6,5,19,5_report_episode_x22_x22 report
7,6,19,6_teilen_beitrag_teilen teilen_jackson
8,7,18,7_corona_regierung_wien_impfung
9,8,17,8_the_to_and_de


In [40]:
text_df

,group_name,denominator,year,messages,date,messages_processed
0,🍃🌺 GᕮᕼᕮIᗰᑎIS 💫 ᗪᕮᖇ GᕮSᑌᑎᗪᕼᕮIT 🌺🍃,4.0,2020.0,‼️‼️❤️‼️‼️❤️‼️‼️ BITTE AN OLIVER JANICH WEITE...,2020-04-01,‼️‼️❤️‼️‼️❤️‼️‼️ bitte an oliver janich weite...
1,USWTV - Videoarchiv,5.0,2021.0,2036 - Das Ende der reptiloiden Schreckensherr...,2021-05-01,2036 - das ende der reptiloiden schreckensherr...
2,Stop-BRD Chat,9.0,2020.0,Danke ich schaue es mir noch mal in Ruhe an. V...,2020-09-01,danke ich schaue es mir noch mal in ruhe an vo...
3,Stop-BRD Chat,8.0,2020.0,Was haltet ihr davon ? 🙌☺️👍 Wi...,2020-08-01,was haltet ihr davon 🙌☺️👍 wie...
4,Stop-BRD Chat,7.0,2020.0,"Hallo alle👋🏼 Hallo Hey, prima k...",2020-07-01,hallo alle👋🏼 hallo hey prima kl...
...,...,...,...,...,...,...
171,GEMEINSAMER ANTI CORONALÜGEN - SIEG CHAT! ZUM ...,5.0,2021.0,🎼 👊✊ Hallo 🔴 Aufkleber zum ausdr...,2021-05-01,🎼 👊✊ hallo 🔴 aufkleber zum ausdr...
172,🧩 SPANDAU steht AUF!,4.0,2021.0,hab hier auf arbeit gerade meinen spaß mit den...,2021-04-01,hab hier auf arbeit gerade meinen spaß mit den...
173,🧩 SPANDAU steht AUF!,5.0,2021.0,Ich habe tatsächlich schon Angebote zu gefälsc...,2021-05-01,ich habe tatsächlich schon angebote zu gefälsc...
174,🧩 SPANDAU steht AUF!,6.0,2021.0,❗️❗️❗️ HILFE BENÖTIGT ❗️❗️Hat einer von euch L...,2021-06-01,❗️❗️❗️ hilfe benötigt ❗️❗️hat einer von euch l...


In [47]:
pd.set_option('display.max_colwidth', None)

In [48]:
data[data['group_name'] == '🍃🌺 GᕮᕼᕮIᗰᑎIS 💫 ᗪᕮᖇ GᕮSᑌᑎᗪᕼᕮIT 🌺🍃' ]['message']

0     <strong>‼️‼️❤️‼️‼️❤️‼️‼️  BITTE AN OLIVER JANICH WEITERLEITEN!    </strong>Lieber Oliver, danke für dieses gutes Video von dir💓 endlich sagt es mal jemand öffentlich das dieser Kanal „unzensiert“ sehr viele  Lügen verbreitet sowie sehr viele negative Dinge über bestimmte Menschen .. oder Menschengruppen !!!  Ich habe vor diesem Kanal schon lange gewarnt!!  Jetzt endlich sagst auch Du es öffentlich ❤️🙏❤️ .. kommt am bei Minute 30:00 des Videos   <a href="https://youtu.be/4-6CwDPwmTE">https://youtu.be/4-6CwDPwmTE</a>  <a href="https://t.me/Geheimnis_Gesundheit">@Geheimnis_Gesundheit</a>  <a href="https://t.me/s/Geheimnis_Gesundheit">https://t.me/s/Geheimnis_Gesundheit</a> - Link ohne telegram-App aufrufbar  Geheimnis der Gesundheit  eure 😇 Angel of Truth       
1                                                                                                                                   ‼️‼️ACHTUNG‼️‼️ <strong> Satanisten Seite mit Fotos von Jay- Z </strong>  Satanistischer Ill

In [33]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import spacy
import pickle

import pyLDAvis
import pyLDAvis.gensim_models

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mangermaier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def preprocessing(df)
    # Remove punctuation
    text_df['messages_processed'] = \
    text_df['messages'].map(lambda x: re.sub('[,\.!?]', '', x))
    # Convert the titles to lowercase
    text_df['messages_processed'] = \
    text_df['messages_processed'].map(lambda x: x.lower())
    # Print out the first rows of papers
    text_df['messages_processed'].head()

0    ‼️‼️❤️‼️‼️❤️‼️‼️  bitte an oliver janich weite...
1    2036 - das ende der reptiloiden schreckensherr...
2    danke ich schaue es mir noch mal in ruhe an vo...
3      was haltet ihr davon         🙌☺️👍        wie...
4     hallo alle👋🏼        hallo        hey prima kl...
Name: messages_processed, dtype: object

In [42]:
stop_words = stopwords.words('german')

In [ ]:
text_df['messages_processed'][3]

In [49]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
lda_data = text_df.messages_processed.values.tolist()
data_words = list(sent_to_words(lda_data))

data_words = remove_stopwords(data_words)

In [ ]:
data_words

In [50]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [51]:
print(corpus[:30])

[(0, 1), (1, 1), (2, 10), (3, 10), (4, 10), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 3), (13, 1), (14, 10), (15, 10), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1)]


In [52]:
from pprint import pprint

# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"fur" + 0.007*"ja" + 0.006*"wurde" + 0.005*"schon" + 0.005*"uber" + '
  '0.004*"danke" + 0.004*"mal" + 0.004*"the" + 0.004*"axel" + 0.004*"gibt"'),
 (1,
  '0.012*"fur" + 0.006*"ja" + 0.004*"wurde" + 0.004*"danke" + 0.004*"mehr" + '
  '0.004*"schon" + 0.004*"the" + 0.003*"mal" + 0.003*"gibt" + 0.003*"immer"'),
 (2,
  '0.015*"fur" + 0.007*"ja" + 0.006*"danke" + 0.006*"schon" + 0.005*"axel" + '
  '0.004*"mehr" + 0.004*"mal" + 0.004*"wurde" + 0.004*"gibt" + 0.003*"immer"'),
 (3,
  '0.013*"fur" + 0.006*"ja" + 0.006*"mal" + 0.006*"schon" + 0.005*"wurde" + '
  '0.004*"mehr" + 0.004*"axel" + 0.004*"danke" + 0.004*"menschen" + '
  '0.004*"immer"'),
 (4,
  '0.011*"fur" + 0.008*"the" + 0.006*"ja" + 0.004*"of" + 0.004*"mal" + '
  '0.004*"and" + 0.004*"wurde" + 0.004*"to" + 0.004*"schon" + 0.004*"danke"'),
 (5,
  '0.011*"fur" + 0.006*"ja" + 0.005*"danke" + 0.005*"schon" + 0.004*"the" + '
  '0.004*"mal" + 0.004*"mehr" + 0.004*"gibt" + 0.003*"menschen" + 0.003*"and"'),
 (6,
  '0.011*"fu

In [ ]:
further_stopwords = (["und", "die", "ich", "der", "fur", "es", "ist", , , , , , , , , , , , , , ])